In [1]:
import pandas as pd
import numpy as np, os
import matplotlib.pyplot as plt

from pathlib import Path
import yaml
import re
import datetime

import pyarrow as pa
import pyarrow.parquet as pq

import gbd_mapping as gm
from vivarium import Artifact

from db_queries import get_ids, get_outputs, get_population, get_covariate_estimates
from get_draws.api import get_draws

import vivarium_helpers as vh
import vivarium_helpers.id_helper as idh
from vivarium_helpers.vph_output.operations import VPHOperator
from vivarium_helpers.utils import convert_to_categorical, constant_categorical, print_memory_usage

!date
!whoami
!pwd

Sun Nov  2 18:17:54 PST 2025
ndbs
/mnt/share/code/ndbs/vivarium_research_alzheimers/results_tables


# Define data directories

In [2]:
# Project directory
project_dir = '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/'

# Output directory to store final results for client
output_dir = Path(r"J:\Project\simulation_science\alzheimers\results_10_31_2025".replace('\\', '/').replace('J:', '/snfs1'))
print(output_dir.exists())
output_dir

True


PosixPath('/snfs1/Project/simulation_science/alzheimers/results_10_31_2025')

In [3]:
# For testing: Run directory containing model 8.3 results for all
# locations
model_run_subdir = 'results/abie_consistent_model_test/united_states_of_america/2025_10_28_08_55_05/'

# Results directory for model 8.3, for testing
results_dirs = project_dir + model_run_subdir + 'results/'

# Artifact for models 8.3 and 8.4
artifact_model_number = '8.3'


In [ ]:
# Model 8.4 results (final run for 10/31 intermediate results) 
# Each batch run contains all locations, 100 random seeds, and 3 or 4 draws
batch_run_dirs = [
    '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.4/model_spec/2025_10_29_20_39_18',
    '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.4/model_spec/2025_10_29_20_41_39', # deduplicated
    '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.4/model_spec/2025_10_29_20_45_13', # 4 draws
    '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.4/model_spec/2025_10_30_14_03_51',
    '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.4/model_spec/2025_10_30_16_32_03',
    '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.4/model_spec/2025_10_30_17_25_38',
    '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.4/model_spec/2025_10_31_01_03_40',
    '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.4/model_spec/2025_10_31_01_09_31',
]

batch_results_dirs = []
for run_dir in batch_run_dirs:
    if run_dir.endswith('2025_10_29_20_41_39'):
        # One batch had to be deduplicated
        results_dir = run_dir + '/deduplicated_results'
    else:
        results_dir = run_dir + '/results'
    batch_results_dirs.append(results_dir)
batch_results_dirs

batch_results_dirs = batch_results_dirs[1:3] # filter for testing
batch_results_dirs


['/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.4/model_spec/2025_10_29_20_39_18/results',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.4/model_spec/2025_10_29_20_41_39/deduplicated_results',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.4/model_spec/2025_10_29_20_45_13/results',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.4/model_spec/2025_10_30_14_03_51/results',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.4/model_spec/2025_10_30_16_32_03/results',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.4/model_spec/2025_10_30_17_25_38/results',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.4/model_spec/2025_10_31_01_03_40/results',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.4/model_spec/2025_10_31_01_09_31/resul

In [5]:
locations = [
    'United States of America',
    'Brazil',
    'China',
    'Germany',
    'Israel',
    'Japan',
    'Spain',
    'Sweden',
    'Taiwan (Province of China)',
    'United Kingdom',
]

def get_results_and_artifact_dicts(
        locations, results_dirs, artifact_model_number, project_dir):

    match results_dirs:
        case str() | Path():
            # Option 1: All locations concatenated in one results
            # directory
            location_to_results_dir = {'all': results_dirs}
        case list():
            # Option 2: One results directory per location
            location_to_results_dir = {
                loc: path for loc, path in zip(locations, results_dirs)}

    location_to_artifact_subdir = {
        loc: loc.lower().replace(' ', '_') for loc in locations}
    artifact_subpaths = [
        f'artifacts/model{artifact_model_number}/' + subdir + '.hdf' 
        for subdir in location_to_artifact_subdir.values()]

    location_to_artifact_path = {
        loc: project_dir + subpath for loc, subpath
        in zip(locations, artifact_subpaths)}

    return location_to_results_dir, location_to_artifact_path

location_to_results_dir, location_to_artifact_path = get_results_and_artifact_dicts(
    locations, results_dirs, artifact_model_number, project_dir
)
# This is needed to assign locations in the model results
location_to_artifact_path

{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf',
 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf',
 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf',
 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf',
 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf',
 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/japan.hdf',
 'Spain': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/spain.hdf',
 'Sweden': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/sweden.hdf',
 'Taiwan (Province of China)': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifact

# Define functions to load simulation results

### First, define some integer or ordered categorical data types for certain columns to save memory and make things work better

E.g., years work better as integers not strings.

In [6]:
# Order locations lexicographically
all_locations = [
    'United States of America',
    'Brazil',
    'China',
    'Germany',
    'Israel',
    'Japan',
    'Spain',
    'Sweden',
    'Taiwan (Province of China)',
    'United Kingdom',
]
all_locations_dtype = pd.CategoricalDtype(sorted(all_locations), ordered=True)

# int16 ranges from -32768 to 32767 (I think), which is sufficient to
# represent all years 2025-2100. uint8 only goes from 0 to 255, which is
# too small.
year_dtype = 'int16'

# Store draws as ints instead of categoricals since we'll be
# concatenating different draws from different results directories
input_draw_dtype = 'int16'

# Order age groups chronologically
age_groups = [f'{age}_to_{age + 4}' for age in range(25, 95, 5)] + ['95_plus']
age_group_dtype = pd.CategoricalDtype(age_groups, ordered=True)

# Order scenarios by complexity
scenarios = ['baseline', 'bbbm_testing', 'bbbm_testing_and_treatment']
scenario_dtype = pd.CategoricalDtype(scenarios, ordered=True)

colname_to_dtype = {
    'location': all_locations_dtype,
    'event_year': year_dtype,
    'age_group': age_group_dtype,
    'scenario': scenario_dtype,
    'input_draw': input_draw_dtype,
}

## Function to load results for all locations and aggregate random seeds

In [7]:
# Create an operator object - treat each random seed as a separate draw,
# and add location to the index
ops = VPHOperator(location_col=True)
# ops.index_cols.extend(['location', 'random_seed'])

def load_sim_output(
        measure,
        results_dict=location_to_results_dir,
        # Pass None to skip filtering locations (when None, must also
        # pass assign_location=False or raw=True)
        location_to_artifact_path=location_to_artifact_path,
        # specify dtypes of certain columns
        colname_to_dtype=colname_to_dtype,
        drop_superfluous_cols=True, # drop redundant or empty columns
        # Sets the 'read_dictionary' key of kwargs, which is passed to
        # pyarrow.parquet.read_table()
        force_parquet_dictionaries=True,
        force_pandas_categoricals=True,
        aggregate_seeds=True,
        assign_location=True,
        raw=False, # Overrides other parameters if True
        **kwargs, # keyword args to pass to .read_parquet
    ):
    """Load simulation output from .parquet files for all locations,
    optionally reducing the size of the data when possible. Returns
    concatenated outputs with a 'location' column added.
    """
    # Override optional transformations if raw=True
    if raw:
        drop_superfluous_cols = False
        force_parquet_dictionaries = False
        force_pandas_categoricals = False
        aggregate_seeds = False
        assign_location = False

    # Determine whether results for all locations are stored in same
    # directory, or if different locations have different results
    # directories
    match location_to_results_dir:
        case {'all': _}:
            all_locations_together = True
        case _:
            all_locations_together = False
    
    if all_locations_together and assign_location and location_to_artifact_path is None:
        raise ValueError(
            "Must provide mapping of artifacts to locations  when" \
            " assign_location=True and all locations are concatenated" \
            " in the simulation outputs."
        )

    dfs = []
    for location, directory in results_dict.items():

        parquet_file_path = Path(directory) / f'{measure}.parquet'
        # Read the Parquet file's schema to get column names and data types
        parquet_schema = pq.read_schema(parquet_file_path)

        if (
            all_locations_together
            and location_to_artifact_path is not None
        ):
            if 'artifact_path' in parquet_schema.names:
                # Filter to locations in list
                location_filter = (
                    'artifact_path',
                    'in',
                    list(location_to_artifact_path.values()),
                )
                user_filters = kwargs.get('filters') # Defaults to None
                kwargs['filters'] = add_parquet_AND_filter(
                    location_filter, user_filters)
                # TODO: Use logging not printing
                print(location_filter)
            else:
                print("'artifact_path' column missing from parquet file."
                      " Not filtering locations.")

        if force_parquet_dictionaries:
            # Read all columns as dictionaries except those containing 
            # floating point values
            kwargs['read_dictionary'] = [
                col.name for col in parquet_schema
                if not pa.types.is_floating(col.type)]

        # Read the parquet file
        df = pd.read_parquet(parquet_file_path, **kwargs)
        print_memory_usage(df, 'after read_parquet')

        if drop_superfluous_cols:
            # Drop redundant columns
            for col1, col2 in [
                ('input_draw', 'input_draw_number'),
                ('entity', 'sub_entity'),
            ]:
                if (col1 in df and col2 in df and df[col1].equals(df[col2])):
                    df.drop(columns=col2, inplace=True)
            # Drop empty columns (e.g., sub-entity)
            for col in df:
                if df[col].isna().all():
                    df.drop(columns=col, inplace=True)
        if colname_to_dtype is not None:
            df = df.astype(
                # Filter to avoid KeyError
                {c: dtype for c, dtype
                 in colname_to_dtype.items() if c in df},
                 # NOTE: If copy-on-write is enabled, copy keyword is
                 # ignored
                 copy=False)
        if force_pandas_categoricals:
            convert_to_categorical(
                df, exclude_cols=colname_to_dtype or (), inplace=True)
        if aggregate_seeds:
            # Use default index and value columns when aggregating
            df = vh.vph_output.operations.marginalize(df, 'random_seed')
        if assign_location:
            if all_locations_together:
                # NOTE: location_to_artifact_path is guaranteed not to
                # be None because assign_location and
                # all_locations_together are both True

                # Find or create a Categorical dtype with all locations
                location_dtype = colname_to_dtype.get(
                    'location',
                    pd.CategoricalDtype(
                        sorted(location_to_artifact_path.keys()), ordered=True)
                )
                # Invert the dictionary so we can map artifact paths to
                # locations
                artifact_path_to_location = {
                    path: loc for loc, path
                    in location_to_artifact_path.items()}
                if 'artifact_path' in df:
                    df['location'] = df['artifact_path'].map(
                        artifact_path_to_location).astype(location_dtype)
                else:
                    # In case the engineers change the DataFrame format
                    # on us...
                    print("'artifact_path' column missing from DataFrame."
                          " Not assigning locations.")
            else:
                # NOTE: location_to_results_dir contains actual
                # locations as keys (not 'all') since
                # all_locations_together is False

                # Find or create a Categorical dtype with all locations
                # to avoid converting back to object dtype.
                location_dtype = colname_to_dtype.get(
                    'location',
                    pd.CategoricalDtype(
                        sorted(location_to_results_dir.keys()), ordered=True)
                )
                df['location'] = location
                df['location'] = df['location'].astype(location_dtype)
        dfs.append(df)
    # TODO: Maybe if assign_location is False and all_locations_together
    # is also False (and there is more than one location?), we should
    # return a dict mapping locations to dataframes (or just a list of
    # dataframes?) instead of concatenating, since it won't be possible
    # to filter the resulting concatenated dataframe by location...
    df = pd.concat(dfs, ignore_index=True)
    return df
    
def add_parquet_AND_filter(new_filter, existing_filters):
    match existing_filters:
        case None:
            # No existing filters -- create a single AND group
            filters = [new_filter]
        case list([tuple((_, _, _)), *_]):
            # Existing filters consist of one AND group -- add the new filter
            filters = [new_filter, *existing_filters]
        case list([list([tuple((_, _, _)), *_]), *_]):
            # Add the filter to each AND group in the outer OR group
            filters = [[new_filter, *and_group] for and_group in existing_filters]
        case _:
            raise ValueError(f"Malformed parquet filter: {existing_filters}")
    return filters

def current_time():
    print(datetime.datetime.now())

## Function to load and concatenate runs from multiple batches

In [8]:
def load_measure_from_batch_runs(
        measure,
        batch_results_dirs,
        locations=locations,
        # Allow loading locations in multiple groups to save memory
        n_location_groups=1,
        colname_to_dtype=colname_to_dtype,
        project_dir=project_dir,
        **kwargs
    ):
    """Load data from multiple batch runs, aggregate random seeds, and
    concatenate.
    """
    # aggregate seeds by default, and warn if False was passed
    if not kwargs.setdefault('aggregate_seeds', True):
        # Documentation for setdefault: If key is in the dictionary,
        # return its value. If not, insert key with a value of default
        # and return default.
        print("Warning: Not aggregating seeds, which may require lots of memory")
    dfs = []
    for results_dir in batch_results_dirs:
        print(results_dir)
        for i in range(n_location_groups):
            location_group = locations[i::n_location_groups]
            # print(location_group)
            location_to_results_dir, location_to_artifact_path = get_results_and_artifact_dicts(
                location_group, results_dir, artifact_model_number, project_dir
            )
            print(location_to_artifact_path)
            df = load_sim_output(
                measure, location_to_results_dir, location_to_artifact_path, colname_to_dtype, **kwargs
            )
            print_memory_usage(df, 'after aggregating seeds and converting dtypes')
            dfs.append(df)
    measure_df = pd.concat(dfs, ignore_index=True)
    print_memory_usage(measure_df, 'total')
    measure_df = measure_df.astype(colname_to_dtype)
    print_memory_usage(measure_df, 'after enforcing dtypes')
    return measure_df

# Load one artifact and define age bins

Looks like this is used in the model scale calculation, I'm not sure
whether it's necessary...

In [9]:
usa_artifact_path = location_to_artifact_path['United States of America']
usa_art = Artifact(usa_artifact_path)
print(usa_art.load('metadata.locations'))

# age_bins is an empty DataFrame with a MultiIndex storing age group data
age_bins = usa_art.load('population.age_bins')
age_dictionary = (
    age_bins
    .reset_index()
    .assign(age_group=lambda df: df['age_group_name'].str.replace(' ', '_'))
    # Filter to ages that actually appear in our sim
    .query("age_start >= 25")
)
# age_dictionary

['United States of America']


# Calculate model scale

In [10]:
start_year = 2022 # We changed from 2025 to 2022 in model 8.4
scale = pd.DataFrame()
for location in locations:
    artifact_path = location_to_artifact_path[location]
    art = Artifact(artifact_path)
    temp = art.load('population.structure').reset_index() 
    temp['location'] = location
    
    df_prev_pop = pd.merge(
        (
            art.load('population.scaling_factor').query("year_start == 2025")
            .rename({2025: start_year}, level='year_start')
            # NOTE: Only works if year_end = year_start + 1
            .rename({2026: start_year + 1}, level='year_end')
        ),
        art.load('population.structure').query("year_start==2025").droplevel(['year_start', 'year_end']),
        left_index=True,
        right_index=True,
        suffixes=['_prev', '_pop']
    )
    prev = ((df_prev_pop.filter(like='draw_').filter(like='_prev')
            * df_prev_pop.filter(like='draw_').filter(like='_pop').values).mean(axis=1)).sum(axis=0)
    # TODO: use draw-specific scale instead of mean
    
    ratio = 100_000 / prev
    print(ratio)

    temp['ratio'] = ratio

    temp = temp.rename(columns={'year_start': 'event_year'})
    temp = temp.merge(age_dictionary, on=['age_start','age_end']) # Is this necessary?
    mini = temp.loc[temp['event_year'] == 2050]
    for year in range(2051, 2100):
        temp = pd.concat([temp, mini.assign(event_year=year)], ignore_index=True)
    scale = pd.concat([scale, temp], ignore_index=True)
scale.head()

0.02068399173333995
0.05127553537168728
0.0055320900654221555
0.04398300673925621
1.8822303322399363
0.030120209603350954
0.15671117018887507
0.6246007133337842
0.4126975201138363
0.09922300617133024


,location,sex,age_start,age_end,event_year,year_end,draw_0,draw_1,draw_2,draw_3,...,draw_494,draw_495,draw_496,draw_497,draw_498,draw_499,ratio,age_group_id,age_group_name,age_group
0,United States of America,Female,25.0,30.0,2021,2022,1.121125e+07,1.166921e+07,1.135183e+07,1.088111e+07,...,1.092046e+07,1.132323e+07,1.166214e+07,1.103869e+07,1.052282e+07,1.089972e+07,0.020684,10,25 to 29,25_to_29
1,United States of America,Female,25.0,30.0,2022,2023,1.121113e+07,1.166051e+07,1.135743e+07,1.088132e+07,...,1.093050e+07,1.132537e+07,1.165253e+07,1.104142e+07,1.053033e+07,1.090925e+07,0.020684,10,25 to 29,25_to_29
2,United States of America,Female,25.0,30.0,2023,2024,1.121362e+07,1.165396e+07,1.136690e+07,1.088590e+07,...,1.094021e+07,1.132919e+07,1.164392e+07,1.104792e+07,1.054449e+07,1.092253e+07,0.020684,10,25 to 29,25_to_29
3,United States of America,Female,25.0,30.0,2024,2025,1.122364e+07,1.165997e+07,1.138827e+07,1.090121e+07,...,1.096114e+07,1.134126e+07,1.164492e+07,1.106502e+07,1.056879e+07,1.094860e+07,0.020684,10,25 to 29,25_to_29
4,United States of America,Female,25.0,30.0,2025,2026,1.125062e+07,1.168256e+07,1.142971e+07,1.093541e+07,...,1.099947e+07,1.137227e+07,1.166691e+07,1.110123e+07,1.060827e+07,1.099398e+07,0.020684,10,25 to 29,25_to_29


# BBBM Test Counts

In [11]:
def dataframe_beutification_and_summarizing(df, measure_name):

    # Add in the scale factor multiplication
    df['event_year'] = df['event_year'].astype(int)
    df = df.merge(
        scale[['location','sex','age_group','ratio','event_year']],
        on=['location','sex','age_group','event_year'])
    df['value'] = df['value'] / df['ratio']    

    # Need to set this up for number and rate to be included 
    df['Metric'] = 'Number'
    df_rate = df.copy()
    df_rate['value'] = df_rate['value'] / 100_000
    df_rate['Metric'] = 'Rate per 100,000'
    df = pd.concat([df, df_rate], ignore_index=True)

    # Renaming, dropping columns, and recategorising
    df = df.rename(columns={'event_year': 'Year ID',
                            'age_group': 'Age',
                            'location': 'Location',
                            'sex':'Sex',
                            'scenario':'Scenario',
                            'sub_entity':'Disease Stage'})
    df['Measure'] = measure_name
    df['Scenario'] = df['Scenario'].cat.rename_categories({
        'baseline': 'Reference',
        'bbbm_testing': 'BBBM Testing Only',
        'bbbm_testing_and_treatment' : 'BBBM Testing and Treatment'
    })
    df['Disease Stage'] = 'Preclinical AD'

    # Now we summarize the data
    df = df.groupby(['Year ID', 'Location', 'Age', 'Sex' , 'Disease Stage' , 'Scenario', 'Measure', 'Metric', 'input_draw']).value.sum().reset_index()
    df = df.groupby(['Year ID', 'Location', 'Age', 'Sex' , 'Disease Stage' , 'Scenario', 'Measure', 'Metric']).value.describe(percentiles=[0.025,0.975]).reset_index()

    df = df.rename(columns={'mean': 'Mean',
                            '2.5%': '95% UI Lower',
                            '97.5%': '95% UI Upper'})

    #Reorder the columns in df 
    column_order = ['Year ID', 'Location', 'Age', 'Sex' , 'Disease Stage' , 'Scenario', 'Measure', 'Metric', 'Mean', '95% UI Lower', '95% UI Upper'] 
    df = df[column_order]

    return df

In [12]:
bbbm_tests = load_sim_output(
    'counts_bbbm_tests',
    )
bbbm_tests.head()

('artifact_path', 'in', ['/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/japan.hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/spain.hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/sweden.hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/taiwan_(province_of_china).hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/a

,age_group,artifact_path,bbbm_test_results,entity,entity_type,event_year,input_draw,measure,scenario,sex,value,location
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,not_tested,bbbm_testing,testing,2025,169,counts_bbbm_tests,baseline,Female,0.0,Japan
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,not_tested,bbbm_testing,testing,2025,169,counts_bbbm_tests,baseline,Male,0.0,Japan
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,not_tested,bbbm_testing,testing,2025,169,counts_bbbm_tests,bbbm_testing,Female,0.0,Japan
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,not_tested,bbbm_testing,testing,2025,169,counts_bbbm_tests,bbbm_testing,Male,0.0,Japan
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,not_tested,bbbm_testing,testing,2025,169,counts_bbbm_tests,bbbm_testing_and_treatment,Female,0.0,Japan


In [ ]:
bbbm_tests_final = dataframe_beutification_and_summarizing(bbbm_tests, 'BBBM Test Counts')

In [14]:
bbbm_tests_final.loc[(bbbm_tests_final['Year ID'] == 2050) & (bbbm_tests_final['Age'] == '65_to_69') & (bbbm_tests_final['Sex'] == 'Female') & (bbbm_tests_final['Metric'] == 'Number')]

## Need to find an old V&V value to compare to. Can't find one quickly.

,Year ID,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
45096,2050,Brazil,65_to_69,Female,Preclinical AD,Reference,BBBM Test Counts,Number,0.000000,0.000000,0.000000
45098,2050,Brazil,65_to_69,Female,Preclinical AD,BBBM Testing Only,BBBM Test Counts,Number,10628.850504,4899.022471,17281.145747
45100,2050,Brazil,65_to_69,Female,Preclinical AD,BBBM Testing and Treatment,BBBM Test Counts,Number,10628.850504,4899.022471,17281.145747
45276,2050,China,65_to_69,Female,Preclinical AD,Reference,BBBM Test Counts,Number,0.000000,0.000000,0.000000
45278,2050,China,65_to_69,Female,Preclinical AD,BBBM Testing Only,BBBM Test Counts,Number,71076.210862,31507.079230,121635.763706
45280,2050,China,65_to_69,Female,Preclinical AD,BBBM Testing and Treatment,BBBM Test Counts,Number,71076.210862,31507.079230,121635.763706
45456,2050,Germany,65_to_69,Female,Preclinical AD,Reference,BBBM Test Counts,Number,0.000000,0.000000,0.000000
45458,2050,Germany,65_to_69,Female,Preclinical AD,BBBM Testing Only,BBBM Test Counts,Number,4669.985415,2510.060321,7341.471717
45460,2050,Germany,65_to_69,Female,Preclinical AD,BBBM Testing and Treatment,BBBM Test Counts,Number,4669.985415,2510.060321,7341.471717
45636,2050,Israel,65_to_69,Female,Preclinical AD,Reference,BBBM Test Counts,Number,0.000000,0.000000,0.000000


In [39]:
# NEED TO ADD IN BBBM TESTS FROM MSLT HERE
!ls $output_dir

2025_10_31_false_positive_bbbm_tests_final.csv	dalys.csv      prevalence.csv
2025_10_31_improper_medication_uses_final.csv	deaths.csv
2025_10_31_susceptible_bbbm_tests_final.csv	incidence.csv


In [42]:
bbbm_tests_susceptible = pd.read_csv(output_dir / '2025_10_31_susceptible_bbbm_tests_final.csv', index_col=0)
bbbm_tests_susceptible

,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
0,2025,Brazil,60_to_65,Female,Susceptible,BBBM Testing and Treatment,BBBM Tests,Number,0.000000e+00,0.000000e+00,0.000000e+00
1,2025,Brazil,65_to_70,Female,Susceptible,BBBM Testing and Treatment,BBBM Tests,Number,0.000000e+00,0.000000e+00,0.000000e+00
2,2025,Brazil,70_to_75,Female,Susceptible,BBBM Testing and Treatment,BBBM Tests,Number,0.000000e+00,0.000000e+00,0.000000e+00
3,2025,Brazil,75_to_80,Female,Susceptible,BBBM Testing and Treatment,BBBM Tests,Number,0.000000e+00,0.000000e+00,0.000000e+00
4,2025,Brazil,60_to_65,Male,Susceptible,BBBM Testing and Treatment,BBBM Tests,Number,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...
6075,2100,United States of America,75_to_80,Female,Susceptible,BBBM Testing and Treatment,BBBM Tests,Number,1.467749e+06,1.362363e+06,1.549757e+06
6076,2100,United States of America,60_to_65,Male,Susceptible,BBBM Testing and Treatment,BBBM Tests,Number,2.317253e+06,2.153712e+06,2.449146e+06
6077,2100,United States of America,65_to_70,Male,Susceptible,BBBM Testing and Treatment,BBBM Tests,Number,1.876975e+06,1.744506e+06,1.983808e+06
6078,2100,United States of America,70_to_75,Male,Susceptible,BBBM Testing and Treatment,BBBM Tests,Number,8.001842e+05,7.437106e+05,8.457287e+05


In [16]:
# prevalence_final.to_csv('/ihme/homes/lutzes/vivarium_research_alzheimers/2025_10_28_prevalence_final.csv')

# CSF and PET Testing

In [17]:
csf_pet_tests = load_sim_output(
    'counts_baseline_tests_among_eligible',
    )
csf_pet_tests.head()

('artifact_path', 'in', ['/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/japan.hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/spain.hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/sweden.hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/taiwan_(province_of_china).hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/a

,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,testing_state,value,location
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,counts_baseline_tests_among_eligible,baseline,Female,not_tested,0.0,Japan
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,counts_baseline_tests_among_eligible,baseline,Female,csf,0.0,Japan
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,counts_baseline_tests_among_eligible,baseline,Female,pet,0.0,Japan
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,counts_baseline_tests_among_eligible,baseline,Female,bbbm,0.0,Japan
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,counts_baseline_tests_among_eligible,baseline,Male,not_tested,0.0,Japan


In [18]:
def dataframe_beutification_and_summarizing(df, measure_name):

    df = df.loc[df.testing_state.isin(['csf','pet'])]
    df['testing_state'] = df['testing_state'].cat.remove_unused_categories()
    df = df.drop(columns=['measure'])

    df = df.rename(columns={'testing_state':'Measure'})

    df['Measure'] = df['Measure'].cat.rename_categories({
        'csf': 'CSF Test Counts',
        'pet': 'PET Test Counts'
    })

    # Add code to make new measures called 'Averted CSF tests' and 'Averted PET tests' by subtracting from baseline
    df_baseline = df.loc[df['scenario'] == 'baseline']
    df_baseline = df_baseline.rename(columns={'value':'baseline_value'})

    df_averted = df.copy().rename(columns={'value':'all_value'})
    df_averted = df_averted.merge(
        df_baseline[['artifact_path', 'entity_type', 'age_group','event_year','location','sex','input_draw','Measure','baseline_value']],
        on=['artifact_path', 'entity_type', 'age_group','event_year','location','sex','input_draw','Measure'])
    df_averted['value'] = df_averted['baseline_value'] - df_averted['all_value']
    df_averted['Measure'] = df_averted['Measure'].cat.rename_categories({
        'CSF Test Counts': 'Averted CSF Test Counts',
        'PET Test Counts': 'Averted PET Test Counts'
    })
    df = pd.concat([df, df_averted], ignore_index=True)

    # Add in the scale factor multiplication
    df['event_year'] = df['event_year'].astype(int)
    df = df.merge(
        scale[['location','sex','age_group','ratio','event_year']],
        on=['location','sex','age_group','event_year'])
    df['value'] = df['value'] / df['ratio']    

    # Need to set this up for number and rate to be included 
    df['Metric'] = 'Number'
    df_rate = df.copy()
    df_rate['value'] = df_rate['value'] / 100_000
    df_rate['Metric'] = 'Rate per 100,000'
    df = pd.concat([df, df_rate], ignore_index=True)

    # Renaming, dropping columns, and recategorising
    df = df.rename(columns={'event_year': 'Year ID',
                            'age_group': 'Age',
                            'location': 'Location',
                            'sex':'Sex',
                            'scenario':'Scenario',
                            # 'sub_entity':'Disease Stage',
                            'testing_state':'Measure'})
    df['Scenario'] = df['Scenario'].cat.rename_categories({
        'baseline': 'Reference',
        'bbbm_testing': 'BBBM Testing Only',
        'bbbm_testing_and_treatment' : 'BBBM Testing and Treatment'
    })
    df['Disease Stage'] = 'MCI due to AD'

    # Now we summarize the data
    df = df.groupby(['Year ID', 'Location', 'Age', 'Sex' , 'Disease Stage' , 'Scenario', 'Measure', 'Metric', 'input_draw']).value.sum().reset_index()
    df = df.groupby(['Year ID', 'Location', 'Age', 'Sex' , 'Disease Stage' , 'Scenario', 'Measure', 'Metric']).value.describe(percentiles=[0.025,0.975]).reset_index()

    df = df.rename(columns={'mean': 'Mean',
                            '2.5%': '95% UI Lower',
                            '97.5%': '95% UI Upper'})

    #Reorder the columns in df
    column_order = ['Year ID', 'Location', 'Age', 'Sex', 'Disease Stage', 'Scenario', 'Measure', 'Metric', 'Mean', '95% UI Lower', '95% UI Upper']
    df = df[column_order]

    return df

In [19]:
csf_pet_tests_final = dataframe_beutification_and_summarizing(csf_pet_tests, 'CSF and PET Test Counts')

In [20]:
csf_pet_tests_final

,Year ID,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
0,2025,Brazil,25_to_29,Female,MCI due to AD,Reference,Averted CSF Test Counts,Number,0.000000,0.000000,0.000000
1,2025,Brazil,25_to_29,Female,MCI due to AD,Reference,Averted CSF Test Counts,"Rate per 100,000",0.000000,0.000000,0.000000
2,2025,Brazil,25_to_29,Female,MCI due to AD,Reference,Averted PET Test Counts,Number,0.000000,0.000000,0.000000
3,2025,Brazil,25_to_29,Female,MCI due to AD,Reference,Averted PET Test Counts,"Rate per 100,000",0.000000,0.000000,0.000000
4,2025,Brazil,25_to_29,Female,MCI due to AD,Reference,CSF Test Counts,Number,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
259195,2060,United States of America,95_plus,Male,MCI due to AD,BBBM Testing and Treatment,Averted PET Test Counts,"Rate per 100,000",0.000193,0.000000,0.000483
259196,2060,United States of America,95_plus,Male,MCI due to AD,BBBM Testing and Treatment,CSF Test Counts,Number,154.709016,53.181224,285.244748
259197,2060,United States of America,95_plus,Male,MCI due to AD,BBBM Testing and Treatment,CSF Test Counts,"Rate per 100,000",0.001547,0.000532,0.002852
259198,2060,United States of America,95_plus,Male,MCI due to AD,BBBM Testing and Treatment,PET Test Counts,Number,222.394210,53.181224,502.804301


In [21]:
csf_pet_tests_final.loc[(csf_pet_tests_final['Year ID'] == 2050) & (csf_pet_tests_final['Age'] == '65_to_69') & (csf_pet_tests_final['Sex'] == 'Female') & (csf_pet_tests_final['Metric'] == 'Number')]

## Again, could not quickly locate an old V&V value to compare to. But appears reasonable?

,Year ID,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
180384,2050,Brazil,65_to_69,Female,MCI due to AD,Reference,Averted CSF Test Counts,Number,0.000000,0.000000,0.000000
180386,2050,Brazil,65_to_69,Female,MCI due to AD,Reference,Averted PET Test Counts,Number,0.000000,0.000000,0.000000
180388,2050,Brazil,65_to_69,Female,MCI due to AD,Reference,CSF Test Counts,Number,1404.178415,1033.631333,1934.645817
180390,2050,Brazil,65_to_69,Female,MCI due to AD,Reference,PET Test Counts,Number,1665.511620,653.333013,2827.859309
180392,2050,Brazil,65_to_69,Female,MCI due to AD,BBBM Testing Only,Averted CSF Test Counts,Number,1014.128855,587.024587,1255.959583
...,...,...,...,...,...,...,...,...,...,...,...
186878,2050,United States of America,65_to_69,Female,MCI due to AD,BBBM Testing Only,PET Test Counts,Number,483.465674,294.914061,623.670719
186880,2050,United States of America,65_to_69,Female,MCI due to AD,BBBM Testing and Treatment,Averted CSF Test Counts,Number,928.254094,435.119106,1155.482960
186882,2050,United States of America,65_to_69,Female,MCI due to AD,BBBM Testing and Treatment,Averted PET Test Counts,Number,1324.695946,638.174689,2281.957980
186884,2050,United States of America,65_to_69,Female,MCI due to AD,BBBM Testing and Treatment,CSF Test Counts,Number,328.756658,106.362448,686.521257


In [22]:
# FIXME: PermissionError -- need to save to different location
# csf_pet_tests_final.to_csv('/ihme/homes/lutzes/vivarium_research_alzheimers/2025_10_28_csf_pet_tests_final.csv')

# Medication Counts

Need initiation, discontinuation and completion

In [23]:
!ls /mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model7.4/united_states_of_america/2025_10_24_16_02_54/results/

counts_baseline_tests_among_eligible.parquet
counts_bbbm_tests.parquet
counts_new_simulants.parquet
counts_newly_eligible_for_bbbm_testing.parquet
deaths.parquet
person_time_alzheimers_disease_and_other_dementias.parquet
person_time_eligible_for_bbbm_testing.parquet
person_time_ever_eligible_for_bbbm_testing.parquet
person_time_treatment.parquet
transition_count_alzheimers_disease_and_other_dementias.parquet
transition_count_treatment.parquet
ylds.parquet
ylls.parquet


In [24]:
medication = load_sim_output(
    'transition_count_treatment',
    )
medication.head()

('artifact_path', 'in', ['/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/japan.hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/spain.hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/sweden.hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/taiwan_(province_of_china).hdf', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/a

265.688959 MB after read_parquet


,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,sub_entity,value,location
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2025,169,transition_count,baseline,Female,waiting_for_treatment_to_full_effect_long,0.0,Japan
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2025,169,transition_count,baseline,Female,waiting_for_treatment_to_full_effect_short,0.0,Japan
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2025,169,transition_count,baseline,Female,full_effect_long_to_waning_effect_long,0.0,Japan
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2025,169,transition_count,baseline,Female,full_effect_short_to_waning_effect_short,0.0,Japan
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2025,169,transition_count,baseline,Female,waning_effect_long_to_no_effect_after_long,0.0,Japan


In [25]:
medication.sub_entity.unique()
# Medication initiation is "waiting_for_treatment_to_full_effect_long" and "waiting_for_treatment_to_full_effect_short"
# Medication discontinuation is "waning_effect_short_to_no_effect_after_short"
# Medication completion is "waning_effect_long_to_no_effect_after_long"

# Note: the discontinuation and completion won't perfectly sum to initiation because some people may die while on treatment

['waiting_for_treatment_to_full_effect_long', 'waiting_for_treatment_to_full_effect_short', 'full_effect_long_to_waning_effect_long', 'full_effect_short_to_waning_effect_short', 'waning_effect_long_to_no_effect_after_long', 'waning_effect_short_to_no_effect_after_short', 'susceptible_to_treatment_to_waiting_for_treat..., 'susceptible_to_treatment_to_no_effect_never_t...]
Categories (8, object): ['full_effect_long_to_waning_effect_long', 'full_effect_short_to_waning_effect_short', 'susceptible_to_treatment_to_no_effect_never_t..., 'susceptible_to_treatment_to_waiting_for_treat..., 'waiting_for_treatment_to_full_effect_long', 'waiting_for_treatment_to_full_effect_short', 'waning_effect_long_to_no_effect_after_long', 'waning_effect_short_to_no_effect_after_short']

In [26]:
medication.loc[medication.sub_entity.isin(['waning_effect_long_to_no_effect_after_long'])]


,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,sub_entity,value,location
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2025,169,transition_count,baseline,Female,waning_effect_long_to_no_effect_after_long,0.0,Japan
12,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2025,169,transition_count,baseline,Male,waning_effect_long_to_no_effect_after_long,0.0,Japan
20,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2025,169,transition_count,bbbm_testing,Female,waning_effect_long_to_no_effect_after_long,0.0,Japan
28,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2025,169,transition_count,bbbm_testing,Male,waning_effect_long_to_no_effect_after_long,0.0,Japan
36,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2025,169,transition_count,bbbm_testing_and_treatment,Female,waning_effect_long_to_no_effect_after_long,0.0,Japan
...,...,...,...,...,...,...,...,...,...,...,...,...
1295964,95_plus,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2060,457,transition_count,baseline,Male,waning_effect_long_to_no_effect_after_long,0.0,China
1295972,95_plus,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2060,457,transition_count,bbbm_testing,Female,waning_effect_long_to_no_effect_after_long,0.0,China
1295980,95_plus,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2060,457,transition_count,bbbm_testing,Male,waning_effect_long_to_no_effect_after_long,0.0,China
1295988,95_plus,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2060,457,transition_count,bbbm_testing_and_treatment,Female,waning_effect_long_to_no_effect_after_long,0.0,China


In [27]:
def dataframe_beutification_and_summarizing(df, measure_name):

    df = df.loc[df.sub_entity.isin(['waiting_for_treatment_to_full_effect_long','waiting_for_treatment_to_full_effect_short'])]
    df['sub_entity'] = df['sub_entity'].cat.remove_unused_categories()
    df = df.drop(columns=['measure'])

    df = df.rename(columns={'sub_entity':'Measure'})
 
    # # Combine the two initiation categories into one before renaming
    # df['Measure'] = df['Measure'].replace({
    #     'waiting_for_treatment_to_full_effect_long': 'Medication Initiation Counts',
    #     'waiting_for_treatment_to_full_effect_short': 'Medication Initiation Counts',
    #     'waning_effect_short_to_no_effect_after_short': 'Medication Discontinuation Counts',
    #     'waning_effect_long_to_no_effect_after_long': 'Medication Completion Counts'
    # })
    # df['Measure'] = df['Measure'].astype('category')

    # Define a Categorical dtype with all 3 categories to preserve
    # Categoricals upon concatenation
    medication_count_dtype = pd.CategoricalDtype(
        ['Medication Initiation Counts', 'Medication Completion Counts', 'Medication Discontinuation Counts'])
    df['Measure'] = df['Measure'].replace({
        'waiting_for_treatment_to_full_effect_long': 'Medication Completion Counts',
        'waiting_for_treatment_to_full_effect_short': 'Medication Discontinuation Counts',
    }).astype(medication_count_dtype)
    # initiation counts = completion counts + discontinuation counts, so
    # we copy the dataframe and rename these both 'initiation' so
    # they'll be added togeter when we do the groupby below
    df_initiation = df.replace(
        {'Measure': {'Medication Completion Counts': 'Medication Initiation Counts',
                     'Medication Discontinuation Counts': 'Medication Initiation Counts'}
        }).astype({'Measure': medication_count_dtype})
    df = pd.concat([df, df_initiation], ignore_index=True)

    # Add in the scale factor multiplication
    df['event_year'] = df['event_year'].astype(int)
    df = df.merge(
        scale[['location','sex','age_group','ratio','event_year']],
        on=['location','sex','age_group','event_year'])
    df['value'] = df['value'] / df['ratio']    

    # Need to set this up for number and rate to be included 
    df['Metric'] = 'Number'
    df_rate = df.copy()
    df_rate['value'] = df_rate['value'] / 100_000
    df_rate['Metric'] = 'Rate per 100,000'
    df = pd.concat([df, df_rate], ignore_index=True)

    # Renaming, dropping columns, and recategorising
    df = df.rename(columns={'event_year': 'Year ID',
                            'age_group': 'Age',
                            'location': 'Location',
                            'sex':'Sex',
                            'scenario':'Scenario'})
    df['Scenario'] = df['Scenario'].cat.rename_categories({
        'baseline': 'Reference',
        'bbbm_testing': 'BBBM Testing Only',
        'bbbm_testing_and_treatment' : 'BBBM Testing and Treatment'
    })
    df['Disease Stage'] = 'Preclinical AD'

    # Now we summarize the data
    df = df.groupby(['Year ID', 'Location', 'Age', 'Sex' , 'Disease Stage' , 'Scenario', 'Measure', 'Metric', 'input_draw'], observed=True).value.sum().reset_index()
    df = df.groupby(['Year ID', 'Location', 'Age', 'Sex' , 'Disease Stage' , 'Scenario', 'Measure', 'Metric'], observed=True).value.describe(percentiles=[0.025,0.975]).reset_index()

    df = df.rename(columns={'mean': 'Mean',
                            '2.5%': '95% UI Lower',
                            '97.5%': '95% UI Upper'})

    #Reorder the columns in df
    column_order = ['Year ID', 'Location', 'Age', 'Sex', 'Disease Stage', 'Scenario', 'Measure', 'Metric', 'Mean', '95% UI Lower', '95% UI Upper']
    df = df[column_order]

    return df

In [28]:
medication_final = dataframe_beutification_and_summarizing(medication, 'Medication Counts')

In [29]:
medication_final.loc[(medication_final['Year ID'] == 2060) & (medication_final['Age'] == '80_to_84') & (medication_final['Sex'] == 'Female') & (medication_final['Metric'] == 'Number')]

## Haven't really validated these but look reasonable? I had to bump up the age group to get non-zero values for medication completion. We should confirm what qualifies as "completion" in the model. 

,Year ID,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
189396,2060,Brazil,80_to_84,Female,Preclinical AD,Reference,Medication Completion Counts,Number,0.000000,0.000000,0.000000
189398,2060,Brazil,80_to_84,Female,Preclinical AD,Reference,Medication Discontinuation Counts,Number,0.000000,0.000000,0.000000
189400,2060,Brazil,80_to_84,Female,Preclinical AD,Reference,Medication Initiation Counts,Number,0.000000,0.000000,0.000000
189402,2060,Brazil,80_to_84,Female,Preclinical AD,BBBM Testing Only,Medication Completion Counts,Number,0.000000,0.000000,0.000000
189404,2060,Brazil,80_to_84,Female,Preclinical AD,BBBM Testing Only,Medication Discontinuation Counts,Number,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
194264,2060,United States of America,80_to_84,Female,Preclinical AD,BBBM Testing Only,Medication Discontinuation Counts,Number,0.000000,0.000000,0.000000
194266,2060,United States of America,80_to_84,Female,Preclinical AD,BBBM Testing Only,Medication Initiation Counts,Number,0.000000,0.000000,0.000000
194268,2060,United States of America,80_to_84,Female,Preclinical AD,BBBM Testing and Treatment,Medication Completion Counts,Number,715.529197,502.804301,821.891645
194270,2060,United States of America,80_to_84,Female,Preclinical AD,BBBM Testing and Treatment,Medication Discontinuation Counts,Number,135.370389,58.015881,188.551613


In [30]:
971.223404 + 109.213877	

1080.437281

In [31]:
medication_final.dtypes

Year ID             int64
Location           object
Age                object
Sex                object
Disease Stage      object
Scenario         category
Measure          category
Metric             object
Mean              float64
95% UI Lower      float64
95% UI Upper      float64
dtype: object

In [32]:
medication_final.loc[(medication_final['Year ID'] == 2060) & (medication_final['Age'] == '75_to_79') & (medication_final['Sex'] == 'Female') & (medication_final['Metric'] == 'Number')]

,Year ID,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
189360,2060,Brazil,75_to_79,Female,Preclinical AD,Reference,Medication Completion Counts,Number,0.000000,0.000000,0.000000
189362,2060,Brazil,75_to_79,Female,Preclinical AD,Reference,Medication Discontinuation Counts,Number,0.000000,0.000000,0.000000
189364,2060,Brazil,75_to_79,Female,Preclinical AD,Reference,Medication Initiation Counts,Number,0.000000,0.000000,0.000000
189366,2060,Brazil,75_to_79,Female,Preclinical AD,BBBM Testing Only,Medication Completion Counts,Number,0.000000,0.000000,0.000000
189368,2060,Brazil,75_to_79,Female,Preclinical AD,BBBM Testing Only,Medication Discontinuation Counts,Number,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
194228,2060,United States of America,75_to_79,Female,Preclinical AD,BBBM Testing Only,Medication Discontinuation Counts,Number,0.000000,0.000000,0.000000
194230,2060,United States of America,75_to_79,Female,Preclinical AD,BBBM Testing Only,Medication Initiation Counts,Number,0.000000,0.000000,0.000000
194232,2060,United States of America,75_to_79,Female,Preclinical AD,BBBM Testing and Treatment,Medication Completion Counts,Number,9060.146727,6642.818358,11791.727784
194234,2060,United States of America,75_to_79,Female,Preclinical AD,BBBM Testing and Treatment,Medication Discontinuation Counts,Number,966.931348,730.033167,1242.506782


In [33]:
# FIXME: PermissionError
# medication_final.to_csv('/ihme/homes/lutzes/vivarium_research_alzheimers/2025_10_28_medication_final.csv')

In [34]:
__name__

'__main__'

In [35]:
!pwd

/mnt/share/code/ndbs/vivarium_research_alzheimers/results_tables


In [36]:
Path().cwd()

PosixPath('/mnt/share/code/ndbs/vivarium_research_alzheimers/results_tables')

# Print time when notebook finishes running

In [37]:
!date

Sun Nov  2 18:47:03 PST 2025
